In [2]:
from pathlib import Path

from ase import units, Atoms
from ase.build import molecule
from ase.io import read, write
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from prefect import flow
from prefect_dask import DaskTaskRunner
from pymatgen.core import Molecule
from pymatgen.io.packmol import PackmolBoxGen

from mlip_arena.models.utils import REGISTRY, MLIPEnum
from mlip_arena.tasks.run import md as MD

In [ ]:
h2 = molecule("H2")
o2 = molecule("O2")
h2o = molecule("H2O")

write("h2.xyz", h2)
write("o2.xyz", o2)
write("h2o.xyz", h2o)

In [ ]:
h2 = Molecule.from_file("h2.xyz")
o2 = Molecule.from_file("o2.xyz")
h2o = Molecule.from_file("h2o.xyz")

In [ ]:
molecules = []

for m, number in zip([h2, o2], [128, 64]):
    molecules.append(
        {
            "name": m.composition.to_pretty_string(),
            "number": number,
            "coords": m,
        }
    )

In [ ]:
tolerance = 2.0
input_gen = PackmolBoxGen(
    tolerance=tolerance,
    seed=1,
)
margin = 0.5 * tolerance

a = 30

packmol_set = input_gen.get_input_set(
    molecules=molecules,
    box=[margin, margin, margin, a - margin, a - margin, a - margin],
)
packmol_set.write_input(".")
packmol_set.run(".")

atoms = read("packmol_out.xyz")
atoms.cell = [a, a, a]
atoms.pbc = True

print(atoms)

write(f'{atoms.get_chemical_formula()}.extxyz', atoms)

In [3]:
atoms = read("H256O128.extxyz")
print(atoms)

Atoms(symbols='H256O128', pbc=True, cell=[30.0, 30.0, 30.0])


In [4]:
nodes_per_alloc = 1
gpus_per_alloc = 4
ntasks = 1

cluster_kwargs = dict(
    cores=1,
    memory="64 GB",
    shebang="#!/bin/bash",
    account="matgen",
    walltime="01:45:00",
    job_mem="0",
    job_script_prologue=[
        "source ~/.bashrc",
        "module load python",
        "source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena",
    ],
    job_directives_skip=["-n", "--cpus-per-task", "-J"],
    job_extra_directives=[
        "-J combustion-water",
        "-q regular",
        f"-N {nodes_per_alloc}",
        "-C gpu",
        f"-G {gpus_per_alloc}",
        f"--exclusive",
        # "--time-min=00:30:00",
        # "--comment=1-00:00:00",
        # "--signal=B:USR1@60",
        # "--requeue",
        # "--open-mode=append"
    ],
    death_timeout=86400
)


cluster = SLURMCluster(**cluster_kwargs)
print(cluster.job_script())
cluster.adapt(minimum_jobs=5, maximum_jobs=50)
client = Client(cluster)

#!/bin/bash

#SBATCH -A matgen
#SBATCH --mem=0
#SBATCH -t 01:45:00
#SBATCH -J combustion-water
#SBATCH -q regular
#SBATCH -N 1
#SBATCH -C gpu
#SBATCH -G 4
#SBATCH --exclusive
source ~/.bashrc
module load python
source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena
/pscratch/sd/c/cyrusyc/.conda/mlip-arena/bin/python -m distributed.cli.dask_worker tcp://128.55.64.22:43419 --name dummy-name --nthreads 1 --memory-limit 59.60GiB --nanny --death-timeout 86400



2024-09-21 02:07:15,149 - distributed.scheduler - ERROR - Task md-8-ae8c6f68afcf406fae6680a18eca9577-1 marked as failed because 4 workers died while trying to run it
2024-09-21 02:07:15,181 - distributed.scheduler - ERROR - Task md-6-04c53616e6ed4c76a754de000182bcad-1 marked as failed because 4 workers died while trying to run it
2024-09-21 03:49:24,062 - distributed.scheduler - ERROR - Task md-9-8d9936131878409688579c984283ffaf-1 marked as failed because 4 workers died while trying to run it


In [5]:
@flow(task_runner=DaskTaskRunner(address=client.scheduler.address), log_prints=True)
def combustion(atoms: Atoms):
    futures = []

    for model in MLIPEnum:
        future = MD.submit(
            atoms=atoms,
            calculator_name=model,
            calculator_kwargs=None,
            ensemble="nvt",
            dynamics="nose-hoover",
            time_step=None,
            ase_md_kwargs=dict(ttime=25 * units.fs, pfactor=None),
            total_time=1000_000,
            temperature=[300, 3000, 3000, 300],
            pressure=None,
            mb_velocity_seed=0,
            traj_file=Path(REGISTRY[model.name]["family"])
            / f"{model.name}_{atoms.get_chemical_formula()}.traj",
            traj_interval=1000,
            restart=True,
        )

        futures.append(future)
    
    return [future.result() for future in futures]

In [ ]:
results = combustion(atoms)

18:15:31.001 | INFO    | prefect.engine - Created flow run 'sepia-buffalo' for flow 'combustion'

18:15:31.004 | INFO    | Flow run 'sepia-buffalo' - View at https://app.prefect.cloud/account/f7d40474-9362-4bfa-8950-ee6a43ec00f3/workspace/d4bb0913-5f5e-49f7-bfc5-06509088baeb/flow-runs/flow-run/7083eb88-b5aa-46f0-9729-e84a6a9f6200

18:15:31.005 | INFO    | prefect.task_runner.dask - Connecting to an existing Dask cluster at tcp://128.55.64.22:43419

18:15:31.012 | INFO    | prefect.task_runner.dask - The Dask dashboard is available at http://128.55.64.22:8787/status

18:15:31.511 | INFO    | Flow run 'sepia-buffalo' - Created task run 'md-2' for task 'md'

...

18:15:32.829 | INFO    | Flow run 'sepia-buffalo' - Submitted task run 'md-1' for execution.

18:15:32.904 | INFO    | Flow run 'sepia-buffalo' - Submitted task run 'md-5' for execution.

18:15:32.942 | INFO    | Flow run 'sepia-buffalo' - Submitted task run 'md-3' for execution.

18:15:32.993 | INFO    | Flow run 'sepia-buffalo' - Submitted task run 'md-7' for execution.

18:15:33.164 | INFO    | Flow run 'sepia-buffalo' - Submitted task run 'md-0' for execution.